# TP1 
### Nicolás CONTRERAS

# Implémentation de méthodes élémentaires pour la classification supervisée : Naive Bayes et classifieur par plus proches voisins

Pour ce TP, nous aurons besoin des modules Python ci-dessous, il vous faut donc évidemment exécuter cette première cellule.

In [2]:
%matplotlib inline
import pandas as pd
import numpy as np
from scipy.stats import multivariate_normal
from sklearn.metrics import confusion_matrix

Le jeu de données [Vertebral Column](https://archive.ics.uci.edu/ml/datasets/Vertebral+Column) permet d'étudier les pathologies d'hernie discale et de Spondylolisthesis. Ces deux pathologies sont regroupées dans le jeu de données en une seule catégorie dite `Abnormale`. 

Il s'agit donc d'un problème de classification supervisée à deux classes :
- Normale (NO) 
- Abnormale (AB)    

avec 6 variables bio-mécaniques disponibles (features).

L'objectif du TP est d'implémenter quelques méthodes simples de classification supervisée pour ce problème.

# Importation des données

> Télécharger le fichier column_2C.dat depuis le site de l'UCI à [cette adresse](https://archive.ics.uci.edu/ml/datasets/Vertebral+Column). 
>
> On peut importer les données sous python par exemple avec la librairie [pandas](https://pandas.pydata.org/pandas-docs/stable/10min.html). Vous pourrez au besoin consulter la documentation de la fonction [read_csv](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html). 
> 
> Le chemin donné dans la fonction `read_csv`est une chaîne de caractère qui spécifie le chemin complet vers le ficher sur votre machine. On peut aussi donner une adresse url si le fichier est disponible en ligne.
>
> Attention à la syntaxe pour les chemins sous Windows doit etre de la forme  `C:/truc/machin.csv`. 
> 
> Voir ce [blog](https://medium.com/@ageitgey/python-3-quick-tip-the-easy-way-to-deal-with-file-paths-on-windows-mac-and-linux-11a072b58d5f) pour en savoir plus sur la "manipulation des chemins" sur des OS variés. 

In [3]:
file_path = 'column_2C.dat'
Vertebral = pd.read_csv(file_path,
                          delim_whitespace =True,
                          header=None)
Vertebral.columns = ["pelvic_incidence",
                                       "pelvic_tilt",
                                       "lumbar_lordosis_angle",
                                       "sacral_slope",
                                       "pelvic_radius",
                                       "degree_spondylolisthesis",
                                       "class"]

> Vérifier à l'aide des méthodes `.head()`  et `describe()` que les données sont bien importées.

In [4]:
Vertebral.head()

,pelvic_incidence,pelvic_tilt,lumbar_lordosis_angle,sacral_slope,pelvic_radius,degree_spondylolisthesis,class
0,63.03,22.55,39.61,40.48,98.67,-0.25,AB
1,39.06,10.06,25.02,29.00,114.41,4.56,AB
2,68.83,22.22,50.09,46.61,105.99,-3.53,AB
3,69.30,24.65,44.31,44.64,101.87,11.21,AB
4,49.71,9.65,28.32,40.06,108.17,7.92,AB


In [5]:
Vertebral.describe()

,pelvic_incidence,pelvic_tilt,lumbar_lordosis_angle,sacral_slope,pelvic_radius,degree_spondylolisthesis
count,310.000000,310.000000,310.000000,310.000000,310.000000,310.000000
mean,60.496484,17.542903,51.930710,42.953871,117.920548,26.296742
std,17.236109,10.008140,18.553766,13.422748,13.317629,37.558883
min,26.150000,-6.550000,14.000000,13.370000,70.080000,-11.060000
25%,46.432500,10.667500,37.000000,33.347500,110.710000,1.600000
50%,58.690000,16.360000,49.565000,42.405000,118.265000,11.765000
75%,72.880000,22.120000,63.000000,52.692500,125.467500,41.285000
max,129.830000,49.430000,125.740000,121.430000,163.070000,418.540000


> Les librairies de Machine Learning telles que `sckitlearn` prennent en entrée des tableau numpy (pas des objets pandas). Créer un tableau numpy que vous nommerez `VertebralVar` pour les features et un vecteur numpy `VertebralClas` pour la variable de classe. Voir par exemple [ici](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_numpy.html#pandas.DataFrame.to_numpy).

In [6]:
Vertebral.columns

Index(['pelvic_incidence', 'pelvic_tilt', 'lumbar_lordosis_angle',
       'sacral_slope', 'pelvic_radius', 'degree_spondylolisthesis', 'class'],
      dtype='object')

In [7]:
features_nom = list(Vertebral.columns)[:-1]
print(features_nom)

['pelvic_incidence', 'pelvic_tilt', 'lumbar_lordosis_angle', 'sacral_slope', 'pelvic_radius', 'degree_spondylolisthesis']


In [8]:
VertebralVar = Vertebral[features_nom].to_numpy()

In [9]:
from itertools import chain
tmp = Vertebral[["class"]].to_numpy()
VertebralClas = np.array(list(chain.from_iterable(tmp)))

In [10]:
VertebralVar

array([[ 63.03,  22.55,  39.61,  40.48,  98.67,  -0.25],
       [ 39.06,  10.06,  25.02,  29.  , 114.41,   4.56],
       [ 68.83,  22.22,  50.09,  46.61, 105.99,  -3.53],
       ...,
       [ 61.45,  22.69,  46.17,  38.75, 125.67,  -2.71],
       [ 45.25,   8.69,  41.58,  36.56, 118.55,   0.21],
       [ 33.84,   5.07,  36.64,  28.77, 123.95,  -0.2 ]])

In [11]:
VertebralClas

array(['AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB',
       'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB',
       'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB',
       'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB',
       'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB',
       'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB',
       'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB',
       'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB',
       'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB',
       'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB',
       'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB',
       'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB',
       'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB',
       'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'A

# Découpage train / test

En apprentissage statistique, classiquement un prédicteur est ajusté sur une partie seulement des données et l'erreur de ce dernier est ensuite évaluée sur une autre partie des données disponibles. Ceci permet de ne pas utiliser les mêmes données pour ajuster et évaluer la qualité d'un prédicteur. Cette problématique est l'objet du prochain chapitre.

> En utilisant la fonction [`train_test_split`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html#sklearn.model_selection.train_test_split) de la librairie [`sklearn.model_selection`](http://scikit-learn.org/stable/modules/classes.html#module-sklearn.model_selection), sélectionner aléatoirement 60% des observations pour l'échantillon d'apprentissage et garder le reste pour l'échantillon de test. 

In [12]:
from sklearn.model_selection import train_test_split

VertebralVar_train, VertebralVar_test, VertebralClas_train, VertebralClas_test = train_test_split(VertebralVar, VertebralClas, train_size=0.6)
len(VertebralVar_train)
ntot = len(VertebralVar_train) + len(VertebralVar_test)  ### longueur totale de l'échantillon -  TO DO ####
ntrain = len(VertebralVar_train)  ### longueur totale de l'échantillon d'apprentissage - TO DO ####
ntest = len(VertebralVar_test) ### longueur totale de l'échantillon de test -TO DO ####

In [13]:
print(f"{ntot=}")
print(f"{ntrain=}")
print(f"{ntest=}")

ntot=310
ntrain=186
ntest=124


Remarque : on peut aussi le faire à la main avec la fonction [`sklearn.utils.shuffle`](https://scikit-learn.org/stable/modules/generated/sklearn.utils.shuffle.html).

# Extraction des deux classes

VertebralClas_train> Extraire les deux sous-échantillons de classes respectives "Abnormale" et "Normale" pour les données d'apprentissage et de test.

In [14]:
VertebralVar_train_AB = np.array([VertebralVar_train[c] for c, d in enumerate(VertebralClas_train) if d == "AB"])
VertebralVar_train_NO = np.array([VertebralVar_train[c] for c, d in enumerate(VertebralClas_train) if d == "NO"])

In [15]:
VertebralVar_test_AB = np.array([VertebralVar_test[c] for c, d in enumerate(VertebralClas_test) if d == "AB"])
VertebralVar_test_NO = np.array([VertebralVar_test[c] for c, d in enumerate(VertebralClas_test) if d == "NO"])

In [16]:
#VertebralVar_test_AB

In [17]:
n_AB = len(VertebralVar_train_AB)
n_NO = len(VertebralVar_train_NO)
print(n_AB)
print(n_NO)

119
67


# Gaussian Naive Bayes

Nous allons ajuster un classifieur naif bayesien sur les données d'apprentissage.

Pour une observation $x \in \mathbb R^6$, la régle du MAP consiste à choisir la catégorie $\hat y (x) = \hat k $ qui maximise (en $k$) 
$$ score_k(x) = \hat \pi_k \prod_{j=1} ^6  \hat f_{k,j}(x_j)   $$
où :
- $k$ est le numéro de la classe ;
- $\hat \pi_k$ est la proportion observée de la classe $k$, 
- $\hat f_{k,j} $ est la densité gaussienne univariée de la classe $k$ pour la variable $j$. Les paramètres de cette loi valent (ajustés par maximum de vraisemblance) :
    - $\hat \mu_{k,j}$ : la moyenne empirique de la variable $X^j$ restreinte à la classe k,
    - $ \hat \sigma^2_{k,j}$ : la variance empirique de la variable $X^j$ restreinte à la classe k.
    
Noter que la fonction $x \mapsto  \prod_{j=1} ^6  f_{k,j}(x_j) $ peut aussi être vue comme une densité gaussienne multidimensionnelle de moyenne $(\mu_{k,1}, \dots, \mu_{k,6})$ et de matrice de covariance diagonale $diag(\hat \sigma^2_{k,1},\dots,\hat  \sigma^2_{k,6})$. Cette remarque évite de devoir calculer le produit de 6 densités univariées, à la place on calcule plus directement la valeur de la densité multidimensionnelle.

Pour calculer la valeur de la densité d'une gaussienne multidimensionnelle en un point $x$ de $\mathbb R ^d$ on peut utililser la fonction [`multivariate_normal`](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.multivariate_normal.html) de la librairie [`scipy.stats`](https://docs.scipy.org/doc/scipy/reference/stats.html). 

On pourra utiliser la fonction `var` de numpy pour calculer le vecteur des variances.

Calcul des moyennes et des variances de chaque variable pour chacun des deux groupes :

In [18]:
#VertebralVar_train_AB

In [19]:
mean_AB = np.mean(VertebralVar_train_AB, axis=0)  
mean_NO = np.mean(VertebralVar_train_NO, axis=0) 

# variances estimées variable par variable pour AB (sur le train) :
var_AB = np.var(VertebralVar_train_AB, axis=0)
# variances estimées variable par variable pour NO (sur le train) :
var_NO = np.var(VertebralVar_train_NO, axis=0)

# on forme les matrices de covariance (matrices diagonales car indep) :
Cov_NB_AB = np.diag(var_AB)
Cov_NB_NO = np.diag(var_NO)

In [20]:
print(f"{mean_AB=}\n")
print(f"{mean_NO=}\n")
print(f"{var_AB=}\n")
print(f"{var_NO=}\n")
print(f"{Cov_NB_AB=}\n")
print(f"{Cov_NB_NO=}")

mean_AB=array([ 64.48310924,  18.30806723,  56.72302521,  46.17554622,
       112.87890756,  33.85      ])

mean_NO=array([ 52.72671642,  13.39149254,  44.38283582,  39.33507463,
       123.7880597 ,   2.58492537])

var_AB=array([273.17884495, 104.60820719, 334.19139085, 175.92257092,
       185.40961981, 748.28483361])

var_NO=array([187.60722205,  49.78195897, 173.89539345, 104.94487276,
        83.9609828 ,  44.74206977])

Cov_NB_AB=array([[273.17884495,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ],
       [  0.        , 104.60820719,   0.        ,   0.        ,
          0.        ,   0.        ],
       [  0.        ,   0.        , 334.19139085,   0.        ,
          0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        , 175.92257092,
          0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ,
        185.40961981,   0.        ],
       [  0.        ,   0.        ,   0.        ,   0. 

Calcul du "score" sur chaque groupe pour chaque element des données test : 

In [21]:
score_NB_test = [
    [n_AB * multivariate_normal.pdf(x, mean=mean_AB, cov=Cov_NB_AB),
    n_NO * multivariate_normal.pdf(x, mean=mean_NO, cov=Cov_NB_NO)]
    for x in VertebralVar_test
]
print(len(score_NB_test))

124


In [22]:
len(VertebralVar_test)

124

In [23]:
pred_NB_test = ['AB' if list2[0] > list2[1] else 'NO' for list2 in score_NB_test]

In [24]:
print(pred_NB_test)

['AB', 'NO', 'AB', 'NO', 'AB', 'AB', 'NO', 'AB', 'NO', 'NO', 'AB', 'AB', 'NO', 'NO', 'AB', 'NO', 'AB', 'AB', 'AB', 'AB', 'AB', 'NO', 'NO', 'AB', 'NO', 'AB', 'NO', 'NO', 'NO', 'NO', 'NO', 'NO', 'NO', 'AB', 'AB', 'NO', 'AB', 'AB', 'NO', 'AB', 'NO', 'AB', 'NO', 'AB', 'AB', 'AB', 'NO', 'NO', 'NO', 'AB', 'AB', 'NO', 'NO', 'AB', 'NO', 'NO', 'AB', 'AB', 'NO', 'AB', 'NO', 'NO', 'NO', 'NO', 'AB', 'NO', 'AB', 'NO', 'NO', 'AB', 'AB', 'AB', 'NO', 'AB', 'NO', 'AB', 'AB', 'AB', 'AB', 'NO', 'NO', 'AB', 'AB', 'NO', 'NO', 'NO', 'AB', 'AB', 'NO', 'NO', 'AB', 'AB', 'AB', 'AB', 'AB', 'NO', 'NO', 'AB', 'AB', 'NO', 'AB', 'NO', 'AB', 'AB', 'AB', 'NO', 'NO', 'AB', 'NO', 'AB', 'AB', 'AB', 'NO', 'NO', 'AB', 'NO', 'AB', 'NO', 'NO', 'NO', 'AB', 'NO', 'NO', 'AB']


La matrice de confusion est une matrice qui synthétise les performances d'une régle de classification. Chaque ligne correspond à une classe réelle, chaque colonne correspond à une classe estimée. La cellule (ligne L, colonne C) contient le nombre d'éléments de la classe réelle L qui ont été estimés comme appartenant à la classe C. Voir par exemple [ici](https://fr.wikipedia.org/wiki/Matrice_de_confusion).

> Evaluer les performances de la méthode sur l'échantillon test. Vous pourrez utiliser la fonction [`confusion_matrix`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html#sklearn.metrics.confusion_matrix) de la librairie [`sklearn.metrics`](http://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics).

In [25]:
from sklearn.metrics import confusion_matrix
cnf_matrix_NB_test = confusion_matrix(VertebralClas_test, pred_NB_test)
#cnf_matrix_NB_test.astype('float') / cnf_matrix_test.sum(axis=1).reshape(-1,1) 

In [26]:
cnf_matrix_NB_test

array([[62, 29],
       [ 1, 32]], dtype=int64)

>  Il existe bien sûr une fonction scikit-learn  pour la méthode Naive Bayes : voir [ici](http://scikit-learn.org/stable/modules/naive_bayes.html). Vérifier que votre prédicteur donne la même réponse de cette fonction.

In [27]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
gnb = GaussianNB()
gnb.fit(VertebralVar_train, VertebralClas_train)
y_pred = gnb.predict(VertebralVar_test)
confusion_matrix(VertebralClas_test, y_pred)

array([[62, 29],
       [ 1, 32]], dtype=int64)

In [28]:
VertebralClas_test

array(['AB', 'NO', 'AB', 'AB', 'AB', 'AB', 'NO', 'AB', 'NO', 'NO', 'AB',
       'AB', 'AB', 'AB', 'AB', 'NO', 'AB', 'AB', 'AB', 'AB', 'AB', 'NO',
       'AB', 'AB', 'NO', 'NO', 'NO', 'AB', 'NO', 'AB', 'AB', 'AB', 'AB',
       'AB', 'AB', 'AB', 'AB', 'AB', 'NO', 'AB', 'AB', 'AB', 'NO', 'AB',
       'AB', 'AB', 'AB', 'NO', 'NO', 'AB', 'AB', 'NO', 'NO', 'AB', 'AB',
       'NO', 'AB', 'AB', 'NO', 'AB', 'AB', 'AB', 'AB', 'NO', 'AB', 'AB',
       'AB', 'NO', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'NO', 'AB', 'AB',
       'AB', 'AB', 'NO', 'AB', 'AB', 'AB', 'NO', 'NO', 'AB', 'AB', 'AB',
       'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'AB', 'NO', 'AB', 'AB',
       'NO', 'AB', 'NO', 'AB', 'AB', 'AB', 'NO', 'NO', 'AB', 'AB', 'AB',
       'AB', 'AB', 'AB', 'NO', 'AB', 'NO', 'AB', 'AB', 'NO', 'AB', 'AB',
       'NO', 'AB', 'AB'], dtype='<U2')

In [29]:
y_pred

array(['AB', 'NO', 'AB', 'NO', 'AB', 'AB', 'NO', 'AB', 'NO', 'NO', 'AB',
       'AB', 'NO', 'NO', 'AB', 'NO', 'AB', 'AB', 'AB', 'AB', 'AB', 'NO',
       'NO', 'AB', 'NO', 'AB', 'NO', 'NO', 'NO', 'NO', 'NO', 'NO', 'NO',
       'AB', 'AB', 'NO', 'AB', 'AB', 'NO', 'AB', 'NO', 'AB', 'NO', 'AB',
       'AB', 'AB', 'NO', 'NO', 'NO', 'AB', 'AB', 'NO', 'NO', 'AB', 'NO',
       'NO', 'AB', 'AB', 'NO', 'AB', 'NO', 'NO', 'NO', 'NO', 'AB', 'NO',
       'AB', 'NO', 'NO', 'AB', 'AB', 'AB', 'NO', 'AB', 'NO', 'AB', 'AB',
       'AB', 'AB', 'NO', 'NO', 'AB', 'AB', 'NO', 'NO', 'NO', 'AB', 'AB',
       'NO', 'NO', 'AB', 'AB', 'AB', 'AB', 'AB', 'NO', 'NO', 'AB', 'AB',
       'NO', 'AB', 'NO', 'AB', 'AB', 'AB', 'NO', 'NO', 'AB', 'NO', 'AB',
       'AB', 'AB', 'NO', 'NO', 'AB', 'NO', 'AB', 'NO', 'NO', 'NO', 'AB',
       'NO', 'NO', 'AB'], dtype='<U2')

# Classifieur par plus proches voisins

Il est préférable d'utiliser la structure de données de type [k-d tree](https://en.wikipedia.org/wiki/K-d_tree) pour effectuer des requêtes de plus proches voisins dans un nuage de points. 

> Contruction du k-d tree pour les données train (pour la métrique euclidienne) :

In [30]:
from sklearn.neighbors import KDTree
tree = KDTree(VertebralVar_train, metric='euclidean')

In [31]:
tree_data, index, tree_nodes, node_bounds = tree.get_arrays()

In [32]:
tree_nodes

array([(  0, 186, 0, 118.91633372), (  0,  93, 0,  66.08913205),
       ( 93, 186, 0, 107.19095578), (  0,  46, 1,  44.82592609),
       ( 46,  93, 1,  52.01734278), ( 93, 139, 1,  69.47665075),
       (139, 186, 1,  88.74809223)],
      dtype=[('idx_start', '<i8'), ('idx_end', '<i8'), ('is_leaf', '<i8'), ('radius', '<f8')])

> Rechercher les 10 plus proches voisins dans les données d'apprentissage du premier point des données de test et afficher les classes de ces observations voisines.

In [33]:
dist, indices_voisins =  tree.query(VertebralVar_test[:1], k=10)
print(dist)
print(indices_voisins)
classes_voisins = np.array([VertebralClas_train[idx] for idx in indices_voisins])[0]
print(classes_voisins)    

[[ 5.57956988  8.08523964  9.01008324 10.07721688 10.15752923 10.33419082
  11.10882532 12.6351573  12.73753508 14.20915902]]
[[ 73   1 173 146 111 168  32  17  80  81]]
['AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'AB' 'NO' 'AB']


In [34]:
type(classes_voisins)

numpy.ndarray

Pour le classifieur par plus proches vosins, la prediction est la classe majoritaire des k plus proches voisins.

> Donner la prédiction pour le premier point de test par vote majoritaire sur ses 10 plus proches voisins 

In [35]:
pred_premier_point = 'AB' if np.count_nonzero(classes_voisins=='AB') > np.count_nonzero(classes_voisins=='NO') else 'NO'

In [36]:
pred_premier_point

'AB'

> Donner la prediction du classifieur ppv pour toutes les données de test. Evaluer la qualité du classifieur.

In [37]:
tree = KDTree(VertebralVar_test, metric='euclidean')
k_class = 10  #nombre de plus proche voisins utilisés
_, indices_voisins =  tree.query(VertebralVar_test, k=10)
classes_voisins = np.array([[VertebralClas_test[idx] for idx in list_values] for list_values in indices_voisins])

pred_kNN_test = np.array(['AB' if np.count_nonzero(list_classes=='AB') > np.count_nonzero(list_classes=='NO') else 'NO' for list_classes in classes_voisins])
cnf_matrix_kNN = confusion_matrix(VertebralClas_test, pred_kNN_test)
#cnf_matrix_kNN.astype('float') / cnf_matrix_kNN.sum(axis=1).reshape(-1,1) 

In [38]:
cnf_matrix_kNN

array([[84,  7],
       [ 8, 25]], dtype=int64)

Il existe bien sûr une fonction scikit-learn pour le classifieur plus proche voisin, voir [ici](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html).

In [39]:
from sklearn.neighbors import KNeighborsClassifier

neigh = KNeighborsClassifier(n_neighbors=k_class)
neigh.fit(VertebralVar_train, VertebralClas_train)

KNeighborsClassifier(n_neighbors=10)

In [40]:
pred = neigh.predict(VertebralVar_test)

In [41]:
confusion_matrix(VertebralClas_test, pred)

array([[76, 15],
       [ 3, 30]], dtype=int64)